## 1. Configuration

In [ ]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests

# Configuration Plotly
import plotly.io as pio
pio.templates.default = "plotly_white"

API_BASE_URL = "http://localhost:8000"

## 2. Chargement des Données

In [ ]:
def load_data():
    try:
        response = requests.get(f"{API_BASE_URL}/accidents?limit=20000")
        df = pd.DataFrame(response.json())
        print(f"Données chargées via API: {len(df)} accidents")
    except:
        df = pd.read_csv('../data/clean/accidents_clean.csv')
        print(f"Données chargées depuis CSV: {len(df)} accidents")
    return df

df = load_data()
df.head()

## 3. Carte Géographique - Densité des Accidents

In [ ]:
# Carte des accidents (si coordonnées disponibles)
if 'lat' in df.columns and 'long' in df.columns:
    # Filtrer les coordonnées valides
    df_map = df[(df['lat'].notna()) & (df['long'].notna())].copy()
    df_map = df_map[(df_map['lat'] >= 41) & (df_map['lat'] <= 51)]  # France métropolitaine
    df_map = df_map[(df_map['long'] >= -5) & (df_map['long'] <= 10)]
    
    # Créer la carte
    fig = px.density_mapbox(
        df_map.sample(min(5000, len(df_map))),  # Limiter pour performance
        lat='lat',
        lon='long',
        radius=10,
        center=dict(lat=46.5, lon=2.5),
        zoom=5,
        mapbox_style="open-street-map",
        title="Carte de Densité des Accidents en France",
        height=600
    )
    
    fig.update_layout(
        title_font_size=18,
        title_x=0.5
    )
    
    fig.show()
    print(f"Carte générée avec {len(df_map)} points")
else:
    print("[ATTENTION] Colonnes 'lat' et 'long' non disponibles")

## 4. Évolution Temporelle Interactive

In [ ]:
# Évolution par année
if 'an' in df.columns:
    temporal_data = df.groupby('an').agg({
        'Num_Acc': 'count',
        'grav': [('graves', lambda x: (x >= 3).sum() if 'grav' in df.columns else 0),
                 ('mortels', lambda x: (x == 4).sum() if 'grav' in df.columns else 0)]
    }).reset_index()
    
    temporal_data.columns = ['Année', 'Total', 'Graves', 'Mortels']
    
    # Créer le graphique
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Évolution du Nombre Total d\'Accidents', 
                       'Évolution des Accidents Graves et Mortels'),
        vertical_spacing=0.15
    )
    
    # Total
    fig.add_trace(
        go.Scatter(
            x=temporal_data['Année'],
            y=temporal_data['Total'],
            mode='lines+markers',
            name='Total',
            line=dict(color='steelblue', width=3),
            marker=dict(size=8)
        ),
        row=1, col=1
    )
    
    # Graves
    fig.add_trace(
        go.Scatter(
            x=temporal_data['Année'],
            y=temporal_data['Graves'],
            mode='lines+markers',
            name='Graves',
            line=dict(color='orange', width=2),
            marker=dict(size=6)
        ),
        row=2, col=1
    )
    
    # Mortels
    fig.add_trace(
        go.Scatter(
            x=temporal_data['Année'],
            y=temporal_data['Mortels'],
            mode='lines+markers',
            name='Mortels',
            line=dict(color='darkred', width=2),
            marker=dict(size=6)
        ),
        row=2, col=1
    )
    
    fig.update_layout(
        height=800,
        title_text="Analyse Temporelle des Accidents",
        title_font_size=20,
        title_x=0.5,
        showlegend=True,
        hovermode='x unified'
    )
    
    fig.update_xaxes(title_text="Année", row=2, col=1)
    fig.update_yaxes(title_text="Nombre d'accidents", row=1, col=1)
    fig.update_yaxes(title_text="Nombre d'accidents", row=2, col=1)
    
    fig.show()

## 5. Distribution de la Gravité

In [ ]:
# Sunburst de la gravité
if 'grav' in df.columns:
    grav_labels = {1: 'Indemne', 2: 'Blessé léger', 3: 'Hospitalisé', 4: 'Tué'}
    df['grav_label'] = df['grav'].map(grav_labels)
    
    grav_counts = df['grav_label'].value_counts().reset_index()
    grav_counts.columns = ['Gravité', 'Count']
    
    fig = px.pie(
        grav_counts,
        values='Count',
        names='Gravité',
        title='Distribution de la Gravité des Accidents',
        hole=0.4,
        color_discrete_sequence=px.colors.sequential.RdBu
    )
    
    fig.update_traces(
        textposition='inside',
        textinfo='percent+label'
    )
    
    fig.update_layout(
        title_font_size=18,
        title_x=0.5,
        height=500
    )
    
    fig.show()

## 6. Heatmap - Accidents par Heure et Jour de la Semaine

In [ ]:
# Heatmap heure x jour
if 'hrmn' in df.columns and 'jour' in df.columns:
    # Extraire l'heure
    df['heure'] = df['hrmn'].astype(str).str.zfill(4).str[:2].astype(int)
    
    # Créer le tableau croisé
    heatmap_data = pd.crosstab(df['heure'], df['jour'])
    
    # Noms des jours
    jour_names = {1: 'Lundi', 2: 'Mardi', 3: 'Mercredi', 4: 'Jeudi', 
                  5: 'Vendredi', 6: 'Samedi', 7: 'Dimanche'}
    heatmap_data.columns = [jour_names.get(col, col) for col in heatmap_data.columns]
    
    fig = px.imshow(
        heatmap_data,
        labels=dict(x="Jour de la semaine", y="Heure", color="Nombre d'accidents"),
        title="Heatmap des Accidents par Heure et Jour",
        color_continuous_scale='YlOrRd',
        aspect='auto'
    )
    
    fig.update_layout(
        title_font_size=18,
        title_x=0.5,
        height=600
    )
    
    fig.show()

## 7. Top Départements - Bar Chart Interactif

In [ ]:
# Top départements
if 'dep' in df.columns:
    top_deps = df['dep'].value_counts().head(15).reset_index()
    top_deps.columns = ['Département', 'Nombre d\'accidents']
    
    fig = px.bar(
        top_deps,
        x='Nombre d\'accidents',
        y='Département',
        orientation='h',
        title='Top 15 Départements - Nombre d\'Accidents',
        color='Nombre d\'accidents',
        color_continuous_scale='Reds'
    )
    
    fig.update_layout(
        title_font_size=18,
        title_x=0.5,
        height=600,
        yaxis={'categoryorder': 'total ascending'}
    )
    
    fig.show()

## 8. Conditions Météo vs Gravité - Grouped Bar Chart

In [ ]:
# Météo vs Gravité
if 'atm' in df.columns and 'grav' in df.columns:
    weather_grav = pd.crosstab(df['atm'], df['grav_label'])
    weather_grav = weather_grav.reset_index()
    
    # Melt pour Plotly
    weather_grav_melted = weather_grav.melt(
        id_vars='atm',
        var_name='Gravité',
        value_name='Count'
    )
    
    fig = px.bar(
        weather_grav_melted,
        x='atm',
        y='Count',
        color='Gravité',
        title='Distribution de la Gravité par Conditions Météorologiques',
        barmode='group',
        labels={'atm': 'Conditions Atmosphériques', 'Count': 'Nombre d\'accidents'}
    )
    
    fig.update_layout(
        title_font_size=18,
        title_x=0.5,
        height=500,
        xaxis_tickangle=-45
    )
    
    fig.show()

## 9. Scatter Plot - Corrélations 3D

In [ ]:
# 3D scatter (si variables appropriées)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

if len(numeric_cols) >= 3:
    # Prendre 3 variables numériques
    x_var = numeric_cols[0] if 'lat' not in numeric_cols else 'lat'
    y_var = numeric_cols[1] if 'long' not in numeric_cols else 'long'
    z_var = numeric_cols[2] if len(numeric_cols) > 2 else numeric_cols[0]
    
    # Sample pour performance
    df_sample = df.sample(min(2000, len(df)))
    
    fig = px.scatter_3d(
        df_sample,
        x=x_var,
        y=y_var,
        z=z_var,
        color='grav_label' if 'grav_label' in df.columns else None,
        title='Visualisation 3D des Accidents',
        opacity=0.7
    )
    
    fig.update_layout(
        title_font_size=18,
        title_x=0.5,
        height=700
    )
    
    fig.show()
else:
    print("[ATTENTION] Pas assez de variables numériques pour le 3D")

## 10. Export des Visualisations

In [ ]:
from pathlib import Path

# Créer dossier outputs
output_dir = Path('../outputs/visualizations')
output_dir.mkdir(parents=True, exist_ok=True)

print("Dossier de sortie créé")
print(f"Répertoire: {output_dir}")
print("\nPour exporter les graphiques:")
print("  fig.write_html('path/to/file.html')")
print("  fig.write_image('path/to/file.png')  # Nécessite kaleido")

## Résumé

Ce notebook a créé **9 types de visualisations** :

1. Carte géographique (densité)
2. Évolution temporelle (séries)
3. Distribution gravité (pie chart)
4. Heatmap (heure x jour)
5. Top départements (bar chart)
6. Météo vs gravité (grouped bars)
7. Scatter 3D (corrélations)

**Utilisation** :
- Toutes les visualisations sont **interactives** (zoom, hover, filters)
- Exportables en HTML pour intégration web
- Réutilisables dans le dashboard Streamlit

**Prochaines étapes** :
- Intégrer dans `streamlit_app.py`
- Ajouter des filtres dynamiques
- Créer un rapport PDF automatisé